In [1]:
# Data analysis packages:
import pandas as pd
import numpy as np
#from datetime import datetime as dt

# Visualization packages:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression #logistic regression
from sklearn import svm #support vector Machine
from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn.naive_bayes import GaussianNB #Naive bayes
from sklearn.tree import DecisionTreeClassifier #Decision Tree
from sklearn.model_selection import train_test_split #training and testing data split
from sklearn import metrics #accuracy measure
from sklearn.metrics import confusion_matrix #for confusion matrix
from sklearn.metrics import f1_score
from time import time
from operator import itemgetter
from scipy.stats import randint
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
from sklearn.cross_validation import  cross_val_score
from __future__ import print_function

import os
import subprocess

from time import time
from operator import itemgetter
from scipy.stats import randint

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
from sklearn.cross_validation import  cross_val_score


filename = 'df_imputed_scaled_OHE_reducedv1.pkl'
df = pd.read_pickle(filename)

X = df[df.columns[:-1]]
y = pd.Series(df['heartdisease'])


#train_X,test_X,train_y,test_y =train_test_split(X,y,test_size=0.33,shuffle = True, random_state=45)

/home/javi_fdez_093/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/javi_fdez_093/.local/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
def report(grid_scores, n_top=3):
    """Report top n_top parameters settings, default n_top=3.

    Args
    ----
    grid_scores -- output from grid or random search
    n_top -- how many to report, of top models

    Returns
    -------
    top_params -- [dict] top parameter settings found in
                  search
    """
    top_scores = sorted(grid_scores,
                        key=itemgetter(1),
                        reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print(("Mean validation score: "
               "{0:.3f} (std: {1:.3f})").format(
               score.mean_validation_score,
               np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

    return top_scores[0].parameters

In [3]:
def run_gridsearch(X, y, clf, param_grid, cv=5):
    """Run a grid search for best Decision Tree parameters.

    Args
    ----
    X -- features
    y -- targets (classes)
    cf -- scikit-learn Decision Tree
    param_grid -- [dict] parameter settings to test
    cv -- fold of cross-validation, default 5

    Returns
    -------
    top_params -- [dict] from report()
    """
    grid_search = GridSearchCV(clf,
                               param_grid=param_grid,
                               cv=cv)
    start = time()
    grid_search.fit(X, y)

    print(("\nGridSearchCV took {:.2f} "
           "seconds for {:d} candidate "
           "parameter settings.").format(time() - start,
                len(grid_search.grid_scores_)))

    top_params = report(grid_search.grid_scores_, 3)
    return  top_params

In [4]:
def run_randomsearch(X, y, clf, para_dist, cv=5,
                     n_iter_search=20):
    """Run a random search for best Decision Tree parameters.

    Args
    ----
    X -- features
    y -- targets (classes)
    cf -- scikit-learn Decision Tree
    param_dist -- [dict] list, distributions of parameters
                  to sample
    cv -- fold of cross-validation, default 5
    n_iter_search -- number of random parameter sets to try,
                     default 20.

    Returns
    -------
    top_params -- [dict] from report()
    """
    random_search = RandomizedSearchCV(clf,
                        param_distributions=param_dist,
                        n_iter=n_iter_search)

    start = time()
    random_search.fit(X, y)
    print(("\nRandomizedSearchCV took {:.2f} seconds "
           "for {:d} candidates parameter "
           "settings.").format((time() - start),
                               n_iter_search))

    top_params = report(random_search.grid_scores_, 3)
    return  top_params

In [6]:
print("-- Grid Parameter Search via 10-fold CV")
from sklearn.model_selection import KFold,StratifiedKFold 
kfold = StratifiedKFold(n_splits=10, random_state=1)


k_range = list(range(1, 101))
# set of parameters to test
param_grid = {"n_neighbors": k_range,
              }

model_KNN = KNeighborsClassifier()
model_KNN_gs = run_gridsearch(X, y, model_KNN, param_grid, cv=kfold)

print("\n-- Best Parameters:")
for k, v in model_KNN_gs.items():
    print("parameter: {:<20s} setting: {}".format(k, v))
    
# test the retuned best parameters
print("\n\n-- Testing best parameters [Grid]...")
final_gs_model = KNeighborsClassifier(**model_KNN_gs)
scores = cross_val_score(final_gs_model, X, y, cv=kfold)
print("mean: {:.3f} (std: {:.3f})".format(scores.mean(),
                                          scores.std()),
                                          end="\n\n" )

-- Grid Parameter Search via 10-fold CV


TypeError: 'StratifiedKFold' object is not iterable

In [7]:
print("-- Random Parameter Search via 10-fold CV")
from sklearn.model_selection import KFold,StratifiedKFold 
kfold = StratifiedKFold(n_splits=10, random_state=1)

# dict of parameter list/distributions to sample
k_range = list(range(1, 101))
# set of parameters to test
param_dist = {"n_neighbors": k_range,
              }

model_KNN = KNeighborsClassifier()
model_KNN_rs = run_randomsearch(X, y, model_KNN, param_dist, cv=kfold,
                         n_iter_search=100)

print("\n-- Best Parameters:")
for k, v in model_KNN_rs.items():
    print("parameter: {:<20s} setting: {}".format(k, v))
    
# test the retuned best parameters
print("\n\n-- Testing best parameters [Grid]...")
final_rs_model = KNeighborsClassifier(**model_KNN_rs)
scores = cross_val_score(final_rs_model, X, y, cv=10)
print("mean: {:.3f} (std: {:.3f})".format(scores.mean(),
                                          scores.std()),
                                          end="\n\n" )




-- Random Parameter Search via 10-fold CV

RandomizedSearchCV took 3.19 seconds for 100 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.788 (std: 0.009)
Parameters: {'n_neighbors': 22}

Model with rank: 2
Mean validation score: 0.785 (std: 0.002)
Parameters: {'n_neighbors': 45}

Model with rank: 3
Mean validation score: 0.784 (std: 0.008)
Parameters: {'n_neighbors': 7}


-- Best Parameters:
parameter: n_neighbors          setting: 22


-- Testing best parameters [Grid]...
mean: 0.797 (std: 0.059)

